

### html比对：找出不同点的文本，在原文本上添加标识和ID，并提取出来

In [37]:
from bs4 import BeautifulSoup

In [38]:
html1_doc = open("./htmls/case5.pdf.html").read()
html2_doc = open("./htmls/case6.pdf.html").read()

# html1_doc = open("./htmls/simplifiedcase3.pdf.html").read()
# html2_doc = open("./htmls/simplifiedcase4.pdf.html").read()

# html1_doc = open("./htmls/case3.pdf.html").read()
# html2_doc = open("./htmls/case4.pdf.html").read()

# html1_doc = open("./合同示例.html").read()
# html2_doc = open("./合同1.html").read()
# html2_doc = open("./空白.html").read()



def extract_body(html_doc):
#     f1 = open("./case3.html","r").read()
    soup = BeautifulSoup(html_doc, "html.parser")
    sidebar = soup.find('div', id='sidebar')
    if sidebar:
        sidebar.extract()
    # [s.extract() for s in soup("head")]
    # print(ss)
#     tt = soup("body")[0].extract()
    return str(soup)

html1_doc = extract_body(html1_doc)
html2_doc = extract_body(html2_doc)

In [39]:
def extract_content_with_format(html_doc):
    html_doc = html_doc.replace(">\n<","><")
    start_index = html_doc.index("<body")
    new_html_doc = []
    for i in range(len(html_doc)):
        new_html_doc.append((html_doc[i],i))
    def keeplr(item):
        if item[0] in [">","<"] and item[1]>=start_index:
            return True
#     flags = list(filter(keeplr,new_html_doc))
    flags = [("<",0)]+list(filter(keeplr,new_html_doc))
    newnew_html_doc = []
    contents = []
    for i in range(0,len(flags)-1):
    #     print(flags[i][0])
    #     print(flags[i+1][0])
    #     print(flags[i+1][1]-flags[i][1])
        if flags[i][0] == ">" and flags[i+1][0] == "<" and (flags[i+1][1]-flags[i][1])>1:
            contents.append(html_doc[flags[i][1]+1:flags[i+1][1]])
            newnew_html_doc.append(flags[i][0]+"%s")
        else:
            newnew_html_doc.append(html_doc[flags[i][1]:flags[i+1][1]])
    newnew_html_doc.append(html_doc[flags[-1][1]:])
    return "".join(newnew_html_doc),contents
formats1,contents1 = extract_content_with_format(html1_doc)
formats2,contents2 = extract_content_with_format(html2_doc)
# contents1

### 获取每个部分的长度

In [40]:
def get_lenofline(text_paras):
    """
    获取每个段落的长度
    """
    lenoflines = []
    return [len(text_paras[i]) for i in range(len(text_paras))]
right_lens = get_lenofline(contents2)
left_lens = get_lenofline(contents1)
# right_lens

In [41]:
def sum_lens(lens):
    new_lens = []
    for i in range(len(lens)):
        new_lens.append(sum(lens[0:i+1]))
    return new_lens

left_indexes = sum_lens(left_lens)
right_indexes = sum_lens(right_lens)
# left_indexes

In [42]:
left_indexes[0:20]

[2, 7, 12]

In [43]:
right_indexes[0:20]

[2, 6, 11, 12]

### 拉成字符串比对

In [44]:
import difflib
class DocxDiff(difflib.HtmlDiff):
    def __init__(self):
        super().__init__()
#         self.MAX_PARA_LEN = config["MAX_PARA_LEN"]
        
    def format_diff(self,fromlines, tolines, fromdesc='', todesc='',context=False, numlines=5, *, charset='utf-8'):
        self._make_prefix()

        # change tabs to spaces before it gets more difficult after we insert
        # markup
        fromlines,tolines = self._tab_newline_replace(fromlines,tolines)

        # create diffs iterator which generates side by side from/to data
        if context:
            context_lines = numlines
        else:
            context_lines = None
        diffs = difflib._mdiff(fromlines,tolines,context_lines,linejunk=self._linejunk,
                      charjunk=self._charjunk)
        
        return list(diffs)

In [4]:
# contents1 = ["2345698","q","56789"]
# contents2 = ["45699","p","6678312"]

In [45]:
import json
differ = DocxDiff()
# 行List对比
# diff_result = differ.format_diff(json.loads(json.dumps(text_paras1)), 
#                                  json.loads(json.dumps(text_paras2))
#                                 ) 

# 字string对比
diff_result = differ.format_diff("".join(contents1), "".join(contents2)) 

# diff_result
# right_diffs = list(map(right_spearte,(filter(right_isok,diff_result))))
# left_diffs = list(map(left_spearte,(filter(left_isok,diff_result))))
# left_diffs,right_diffs = left_right_process(diff_result)

In [68]:
# differ.format_diff("pre09\x061234\x06\x061234\x06back", "123406\x061234\x067878") 
differ.format_diff(["2213"],["345","345"])

[((1, '\x00-2213\x01'), (1, '\x00+345\x01'), True),
 (('', '\n'), (2, '\x00+345\x01'), True)]

In [34]:
differ.format_diff("2211", "345345")  # 2211 vs 345 345
left_lens = [4]
right_lens = [3,6]


[((1, '\x00-2\x01'), (1, '\x00+3\x01'), True),
 ((2, '\x00-2\x01'), (2, '\x00+4\x01'), True),
 ((3, '\x00-1\x01'), (3, '\x00+5\x01'), True),
 ((4, '\x00-1\x01'), (4, '\x00+3\x01'), True),
 (('', '\n'), (5, '\x00+4\x01'), True),
 (('', '\n'), (6, '\x00+5\x01'), True)]

In [55]:
diff_res = differ.format_diff("221378", "345345")  # 2211 vs 34534 5
diff_res

[((1, '\x00-2\x01'), ('', '\n'), True),
 ((2, '\x00-2\x01'), ('', '\n'), True),
 ((3, '\x00-1\x01'), ('', '\n'), True),
 ((4, '3'), (1, '3'), False),
 ((5, '\x00-7\x01'), (2, '\x00+4\x01'), True),
 ((6, '\x00-8\x01'), (3, '\x00+5\x01'), True),
 (('', '\n'), (4, '\x00+3\x01'), True),
 (('', '\n'), (5, '\x00+4\x01'), True),
 (('', '\n'), (6, '\x00+5\x01'), True)]

In [56]:
%store diff_res

Stored 'diff_res' (list)


In [9]:
len("\x01")

2

### 罗列比对左右的内容

In [46]:
def listdiff(diffs):

    Ldiffs = []
    new_diffs = []
    for item in diffs:
        l = item[0][1]
        r = item[1][1]
        if "\x00-" in l and "\00+" in r:
            l = l.replace("\x00-", "").replace("\x01", "")  # \x04 mod
            r = r.replace("\x00+", "").replace("\x01", "")  # \x04 mod
            new_diffs.append((l, r, "\x04"))
        elif "\x00-" in l:
            l = l.replace("\x00-", "").replace("\x01", "")  # \x03  del
            new_diffs.append((l, r, "\x03"))
        elif "\x00+" in r:
            r = r.replace("\x00+", "").replace("\x01", "")  # \x02 add
            new_diffs.append((l, r, "\x02"))
        else:
            new_diffs.append((l, r, "0"))
        cur = 0
    #     print(new_diffs)
    cur = None
    for d in new_diffs:
        c2 = d[2]
        c0 = d[0]
        c1 = d[1]
        #         if c2 in ["\x02","\x03","\x04"]:
        if cur != c2:
            cur = c2
            if cur == "\x02":
                #                 Ladds.append([c0,c1,"add"])
                Ldiffs.append([c0, c1, "add", len(Ldiffs)])
            elif cur == "\x03":
                #                 Ldels.append([c0,c1,"del"])
                Ldiffs.append([c0, c1, "del", len(Ldiffs)])
            elif cur == "\x04":
                #                 Lchgs.append([c0,c1,"mod"])
                Ldiffs.append([c0, c1, "mod", len(Ldiffs)])
            else:
                Ldiffs.append([c0, c1, "0", len(Ldiffs)])
        else:
            if cur == "\x02":
                #                 Ladds[-1][0] += c0
                #                 Ladds[-1][1] += c1
                Ldiffs[-1][0] += c0
                Ldiffs[-1][1] += c1
            elif cur == "\x03":
                #                 Ldels[-1][0] += c0
                #                 Ldels[-1][1] += c1
                Ldiffs[-1][0] += c0
                Ldiffs[-1][1] += c1
            elif cur == "\x04":
                #                 Lchgs[-1][0] += c0
                #                 Lchgs[-1][1] += c1
                Ldiffs[-1][0] += c0
                Ldiffs[-1][1] += c1
            else:
                Ldiffs[-1][0] += c0
                Ldiffs[-1][1] += c1
    return Ldiffs
Ldiffs = listdiff(diff_result)
Ldiffs

[['12121', '12121', '0', 0],
 ['1', '\n', 'del', 1],
 ['21', '21', '0', 2],
 ['2', '3', 'mod', 3],
 ['1', '1', '0', 4],
 ['\n', '5', 'add', 5]]

In [47]:
def append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1):
#     print(c1_index)
    Ldiffs[-1][0][-1] += c0  # left
    Ldiffs[-1][1][-1] += c1  # right
    if c0_index in left_indexes:
        Ldiffs[-1][0].append("")
    if c1_index in right_indexes:
        Ldiffs[-1][1].append("")
    

def listdiff(diffs,left_indexes=[],right_indexes=[]):
    """
    嵌套List
    外层List的index对应diff_id
    字符级别diff转为line级别的，合同相同类型的char
    """
    Lchgs = []
    Ladds = []
    Ldels = []
    Ldiffs = []
    
    Ldiffs = []
    new_diffs = []
    
    # 格式修改
    for item in diffs:
        l_index = item[0][0]
        r_index = item[1][0]
        l = item[0][1]  
        r = item[1][1]
        if "\x00-" in l and "\00+" in r:
            l = l.replace("\x00-","").replace("\x01","")  # \x04 mod
            r = r.replace("\x00+","").replace("\x01","")  # \x04 mod
            le = (l_index,l)
            re = (r_index,r)
            new_diffs.append((le,re,"\x04"))
        elif "\x00-" in l:
            l = l.replace("\x00-","").replace("\x01","") # \x03  del
            le = (l_index,l)
            re = (r_index,r)
            new_diffs.append((le,re,"\x03"))
        elif "\x00+" in r:
            r = r.replace("\x00+","").replace("\x01","")  # \x02 add
            le = (l_index,l)
            re = (r_index,r)
            new_diffs.append((le,re,"\x02"))
        else:
            le = (l_index,l)
            re = (r_index,r)
            new_diffs.append((le,re,"0"))
        cur = 0
#     print(new_diffs)
    cur = None
    
    # 再次格式修改  chars-->lines
    for d in new_diffs:
        c2 = d[2]
        
        c0_index = d[0][0]
        c0 = d[0][1]
        c1_index = d[1][0]
        c1 = d[1][1]
#         if c2 in ["\x02","\x03","\x04"]:
        # 外层List 类型发生变化则外层List append
        if cur != c2:
            cur = c2
            if cur == "\x02":
#                 Ladds.append([c0,c1,"add"])
                Ldiffs.append([[c0],[c1],"add",len(Ldiffs)])
            elif cur == "\x03":
#                 Ldels.append([c0,c1,"del"])
                Ldiffs.append([[c0],[c1],"del",len(Ldiffs)])
            elif cur == "\x04":
#                 Lchgs.append([c0,c1,"mod"])
                Ldiffs.append([[c0],[c1],"mod",len(Ldiffs)])
            else:
                Ldiffs.append([[c0],[c1],"0",len(Ldiffs)])
        # 内层List
        else:  # cur == c2
#             if c0_index in left_indexes:
#                 pass
            
#             if c1_index in right_indexes:
#                 pass
            
            if cur == "\x02":
#                 Ladds[-1][0] += c0 
#                 Ladds[-1][1] += c1
#                 Ldiffs[-1][0][-1] += c0 
#                 Ldiffs[-1][1][-1] += c1
                append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1)
            elif cur == "\x03":
#                 Ldels[-1][0] += c0 
#                 Ldels[-1][1] += c1
#                 Ldiffs[-1][0][-1] += c0 
#                 Ldiffs[-1][1][-1] += c1
                append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1)
            elif cur == "\x04":
#                 Lchgs[-1][0] += c0 
#                 Lchgs[-1][1] += c1
#                 Ldiffs[-1][0][-1] += c0 
#                 Ldiffs[-1][1][-1] += c1
                append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1)
            else:
#                 Ldiffs[-1][0][-1] += c0 
#                 Ldiffs[-1][1][-1] += c1
                append_chars(Ldiffs,left_indexes,right_indexes,c0_index,c1_index,c0,c1)
    return Ldiffs

Ldiffs = listdiff(diff_result,left_indexes,right_indexes)
Ldiffs

[[['', '12121', ''], ['', '1212', '1'], '0', 0],
 [['1'], ['\n'], 'del', 1],
 [['21'], ['21'], '0', 2],
 [['2'], ['3'], 'mod', 3],
 [['1'], ['1'], '0', 4],
 [['\n'], ['5'], 'add', 5]]

In [366]:
a = [1,2,3]
a[-1:]

[3]

In [384]:
a = ((1,2),(3,4),5)
a[0][0]

1

In [449]:
# 将有变化的地方过滤出来
def isdiff(item):
    if item[2] != "0":
        return True
diff_lines = list(filter(isdiff,Ldiffs))
# len(diff_lines)

In [48]:
# len(diff_lines[1][0])  # 57
diff_lines[1][0]

NameError: name 'diff_lines' is not defined

In [369]:
def topK(Ldiffs,dt="add",K=3):
    res_lines = []
    newLdiffs = sorted(Ldiffs, key=lambda x: (x[2],len(x[0])), reverse=True)
    print(len(newLdiffs))
    res_diffs = []
    for i in range(min(K,len(newLdiffs))):
        cur = newLdiffs[i]
        if cur[2]==dt:
            index = cur[3]
            if index >0 and Ldiffs[index-1][2] == "0":
                pre = Ldiffs[index-1][0]
            else:
                pre = ""
            if index < len(Ldiffs)-1 and Ldiffs[index+1][2] == "0":  # 不变项
                back = Ldiffs[index+1][0] 
            else:
                back = ""
#             res_diffs.append([[pre[-10:],cur[0],back[0:10]],[pre[-10:],cur[1],back[0:10]]])
            line = {"from":(pre[-10:],cur[0],back[0:10]),"to":(pre[-10:],cur[1],back[0:10])}
            res_lines.append(line)
    return res_lines
topK(Ldiffs)

6


[{'from': ('1', '\n', ''), 'to': ('1', '5', '')}]

In [370]:
# 统计三类diff的个数
def diff_stat(diff_lines):
    add_count = 0
    delete_count = 0
    change_count = 0
    for line in diff_lines:
        if line[2] == "mod":
            change_count +=1
        if line[2] == "add":
            add_count +=1
        if line[2] == "del":
            delete_count +=1
#     stat = {"add": add_count, "delete": delete_count, "change": change_count}
    return {"add": add_count, "delete": delete_count, "change": change_count}

diff_stat(diff_lines)

{'add': 1, 'delete': 1, 'change': 1}

In [451]:
def left_spearte(item):
    l = item[0][1]
    r = item[1][1]
    if "\x00-" in l and "\00+" in r:
        l = l.replace("\x00-","\x04").replace("\x01","\x04")  # \x04 mod
    elif "\x00-" in l:
        l = l.replace("\x00-","\x03").replace("\x01","\x03") # \x03  del
    elif "\x00+" in l:
        l = l.replace("\x00+","\x02").replace("\x01","\x02")  # \x02 add
    if "\x05" in l:
        l = "\x05"
    return (item[0][0]-1,l)

def right_spearte(item):
    l = item[0][1]
    r = item[1][1]
    if "\x00-" in l and "\00+" in r:
        r = r.replace("\x00+","\x04").replace("\x01","\x04")
    elif "\x00-" in r:
        r = r.replace("\x00-","\x03").replace("\x01","\x03")
    elif "\x00+" in r:
        r = r.replace("\x00+","\x02").replace("\x01","\x02")
    if "\x05" in r:
        r = "\x05"
    return (item[1][0]-1,r)

def left_isok(item):
#     return True
    return item[0][0] != ""
def right_isok(item):
#     return True
    return item[1][0] != ""

def processdiffchars(diff_chars):
    """
    1. 连续
    2. left没有add
    3. right没有delete
    """
    cursor = 0
    left_res= []
    right_res = []
    left_tmp = ""
    right_tmp = ""
    while(cursor<len(diff_chars) and (not left_isok(diff_chars[cursor]) or not right_isok(diff_chars[cursor]))):
#         print(cursor)
        if left_isok(diff_chars[cursor]):
            left_res.append(left_spearte(diff_chars[cursor]))
        else:
            left_tmp = "\x02\x02\x02\x02"
        if right_isok(diff_chars[cursor]):
            right_res.append(right_spearte(diff_chars[cursor]))
        else:
            right_tmp = "\x03\x03\x03\x03"
        cursor += 1
    if cursor < len(diff_chars):
        left_res.append(left_spearte(diff_chars[cursor]))
        right_res.append(right_spearte(diff_chars[cursor]))
        left_res[-1] = (left_res[-1][0], left_tmp+ left_res[-1][1]) 
        right_res[-1] = (right_res[-1][0], right_tmp +right_res[-1][1]) 
        for i in range(cursor+1,len(diff_chars)):
            if left_isok(diff_chars[i]):
                left_res.append(left_spearte(diff_chars[i]))
            else:
                if not left_res[-1][1].endswith("\x02\x02\x02\x02"):
                    left_res[-1] = (left_res[-1][0],left_res[-1][1] + "\x02\x02\x02\x02") 

            if right_isok(diff_chars[i]):
                right_res.append(right_spearte(diff_chars[i]))
            else:
                if not right_res[-1][1].endswith("\x03\x03\x03\x03"):
                    right_res[-1] = (right_res[-1][0], right_res[-1][1]+ "\x03\x03\x03\x03" )
    return left_res,right_res
left_res,right_res = processdiffchars(diff_result)
# right_res

In [372]:
# left_res

In [455]:
from functools import reduce
def mergeline(item0,item1):
    left = item0[1]
    right = item1[1]
#     print(left[-1])
#     print(right[0])
#     for 
    if len(right)>1 and len(left)>1:
#         if left[-1] == right[0] and left[-2]!="\x05" and right[1]!="\x05":
        if left[-1] == right[0]:
            return (0,left[0:-1]+right[1:])
    return (0,left+right)


# def process_table(formats,lines):
#     for i in lines

diff_prefix = "%s@%s" # left-diff_id@sub_id 


def merge_addid(diffs,lenoflines,K=3,dire="left"):
    """
    diffs:
    lens: 
    return lines
    """
    chgs = """<span class=\""""+ dire +"""-mod-text" id=%s>"""
    adds = """<span class=\""""+ dire +"""-add-text" id=%s>"""
    dels = """<span class=\""""+ dire +"""-del-text" id=%s>"""
    ends = """</span>"""
    start = 0
    diffid = 0
    res_lines = []
    mid_lines = []
    cur_diff = 0
    diffsubid = 0
    for index in range(len(lenoflines)):
        """
        先按格式分，再按diff分
        相同类型相连则算一个diff_id
        同一个diff_id下， html标签不同，则sub_diff_id不同
        """
        chars = []
        cur_line_diffs = diffs[start:start+lenoflines[index]]
        if len(cur_line_diffs)>0:
            cur_line = reduce(mergeline, cur_line_diffs)[1]
            cur_line = cur_line.replace("\x02\x02","\x02").replace("\x03\x03","\x03").replace("\x04\x04","\x04")
            mid_lines.append(cur_line)
            f_cs = []
            cur = 0
#             print(cur_line)
            if len(cur_line)>=1:
                c = cur_line[0]
                if c in ["\x02","\x03","\x04"]:
                    if cur_diff == 0:
                        diffid += 1
                        diffsubid = 0
                    else:
                        if c != cur_diff:
                            diffid += 1
                            diffsubid = 0
                        else:
                            diffsubid += 1
                    cur = c
                    cur_diff = c
                    ID = str(diffid)+"@"+str(diffsubid)
                    c = c.replace("\x02",adds % ID).replace("\x03",dels% ID).replace("\x04",chgs% ID)
                else:
                    cur_diff = 0
    #             print(f_cs) 
                f_cs.append(c)
                for i in range(1,len(cur_line)):
                    c = cur_line[i]
                    if c in ["\x02","\x03","\x04"]:
    #                     print(c)
                        if c != cur:
                            diffid += 1
                            cur = c
                            cur_diff = c
                            diffsubid = 0
                            ID = str(diffid)+"@"+str(diffsubid)
                            c = c.replace("\x02",adds % ID).replace("\x03",dels% ID).replace("\x04",chgs% ID)
                        else:
                            cur = 0
                            c = ends
#                     else:
#                         cur_diff = 0
                    f_cs.append(c)
                res_line = "".join(f_cs)
                if len(cur_line)>1:
#                     pass
#                     print(cur_line[-1])
                    if cur_line[-1] not in ["\x02","\x03","\x04"]:
                        cur_diff = 0
            else:
                res_line = ""
        else:
            res_line = ""
        res_lines.append(res_line)
        start += lenoflines[index]
    return res_lines,mid_lines

#     for i in range(1,len(diffs)):
# res_lines,mid_lines = merge_addid(right_res,right_lens)
res_lines,mid_lines = merge_addid(left_res,left_lens)
# res_lines

In [456]:
# formats

In [457]:
def add_formats(formats,new_lines):
    fs = formats.split("%s")
    finals = []
    for i in range(len(fs) - 1):
        finals.append(fs[i])
        finals.append(new_lines[i])
    finals.append(fs[-1])
    #     return formats % tuple(new_lines)
    return "".join(finals)
# res = add_formats(formats2,res_lines)
res = add_formats(formats1,res_lines)

In [484]:
from bs4 import BeautifulSoup
import re
def extract_spans(html_doc):

    soup = BeautifulSoup(html_doc, "html.parser")
#     soup.find_all(class_=re.compile("itl"))
#     text = soup.find_all('span',class_="left-del-text")
    text = soup.find_all('span',class_=re.compile("left-"))
#     print(text)
    return text
#     if sidebar:
#         sidebar.extract()
#     return str(soup)

spans = extract_spans(res)
# for sp in spans:
#     print(sp)
#     if "left" in sp:
#         print(sp)

In [458]:
open("./case34_diff.html","w").write(res)

4736647

In [255]:
a /= 2

TypeError: unsupported operand type(s) for /=: 'list' and 'int'